<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d57861053f15d5ff46c23b62ae177eb910b0f9f9cbde44039b55ca959b8d1e88
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-07 15:00:53
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -10.08 K (-0.07%)
Current PnL: -20.56 L (-13.64%)
CY Booked + Current PnL: -9.12 L (-6.05%)
-------------------
Total profit:  1.39 L
Total loss:  -21.95 L
-------------------
Total Booked + Current PnL: 17.86 L (14.44%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.11%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.06 L (62.44%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,62.0,H-MC,13.72,115010.0,14506.0,14606.0,-0.07,14.43,12.70,28.96,79.0,0.99,0.82,53.30,MH,ATH,METALS
14,BLUESTARCO,2080.00,10.55,51.0,H-MC,2.97,190930.0,26260.0,17069.0,0.87,15.95,8.94,26.31,89.0,1.54,1.36,25.20,X40N,NTT,AC
77,TTKPRESTIG,770.00,93.17,39.0,M-SC,1.35,83264.0,-17513.0,17610.0,0.03,-17.38,21.15,0.09,245.0,-0.99,0.59,9.39,OX40N,NTT,DURABLES
19,CIPLA,1662.74,-22.18,43.0,H-LC,3.90,207254.0,2754.0,20539.0,-0.02,1.35,9.91,11.39,10.0,0.13,1.48,10.42,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.29,53.0,H-LC,3.34,216622.0,6256.0,22529.0,0.99,2.97,10.40,13.68,37.0,0.28,1.54,19.99,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,STARHEALTH,761.0,16.61,69.0,H-SC,5.88,259570.0,-21553.0,154418.0,4.30,-7.67,59.49,47.26,144.0,-0.14,1.85,39.07,XY24,NTT,INSURANCE
58,REPCOHOME,880.0,-59.02,58.0,H-SC,3.40,233604.0,-52901.0,310226.0,4.19,-18.46,132.80,89.82,134.0,-0.17,1.66,23.59,XY24,NTT,FINANCE
75,TITAGARH,1548.0,-1.45,59.0,H-SC,4.82,210455.0,-31722.0,142478.0,2.69,-13.10,67.70,45.74,158.0,-0.22,1.50,36.88,XY24,NTT,ENGINEERING
61,SAMMAANCAP,326.0,-185.23,69.0,M-SC,12.61,147159.0,-2061.0,146247.0,2.23,-1.38,99.38,96.62,208.0,-0.01,1.05,59.26,XY25,NTT,FINANCE
44,JIOFIN,387.0,-1.80,52.0,H-LC,12.74,266944.0,-421.0,65882.0,1.36,-0.16,24.68,24.48,48.0,-0.01,1.90,54.73,XY24,BTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HAPPSTMNDS,1488.71,-25.89,17.0,H-SC,11.91,79928.0,-48237.0,158265.0,-4.70,-37.64,198.01,85.85,132.0,-0.30,0.57,0.00,AR,ATH,IT
46,KPIGREEN,731.05,-8.64,29.0,H-SC,6.74,108814.0,-16483.0,75408.0,-2.53,-13.16,69.30,47.03,141.0,-0.22,0.77,36.24,MH,ATH,POWER
55,RAJOOENG,143.10,-41.44,41.0,H-SC,16.41,88710.0,-13790.0,54388.0,-2.02,-13.45,61.31,39.61,114.0,-0.25,0.63,8.34,AR,ATH,MISC
82,VALIANTORG,838.00,-300.25,37.0,H-SC,2.76,111248.0,-52357.0,173669.0,-1.98,-32.00,156.11,74.15,139.0,-0.30,0.79,42.60,XR,NTT,CHEMICALS
69,SURYODAY,240.00,62.28,61.0,H-SC,6.64,148196.0,-30875.0,96609.0,-1.96,-17.24,65.19,36.71,135.0,-0.32,1.06,47.04,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1228.55,65.0,M-SC,20.86,180630.0,-513.0,45988.0,-1.64,-0.28,25.46,25.10,235.0,-0.01,1.29,44.76,XY24,NTT,HEALTHCARE
37,INDIAMART,4810.62,-55.57,32.0,H-SC,9.32,124248.0,912.0,130709.0,-1.33,0.74,105.20,106.72,119.0,0.01,0.88,24.29,AR,ATH,MISC
35,ICICIPRULI,790.00,-20.65,51.0,H-MC,2.04,136448.0,660.0,42094.0,0.55,0.49,30.85,31.48,107.0,0.02,0.97,12.66,X40,ATH,INSURANCE
86,WIPRO,420.00,-14.25,45.0,M-LC,5.71,150796.0,-149.0,109599.0,0.45,-0.10,72.68,72.51,53.0,-0.00,1.07,5.92,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,65.0,H-SC,13.51,131220.0,-11259.0,30784.0,-0.59,-7.90,23.46,13.70,163.0,-0.37,0.93,54.88,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.17,39.0,M-SC,1.35,83264.0,-17513.0,17610.0,0.03,-17.38,21.15,0.09,245.0,-0.99,0.59,9.39,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,50.0,H-SC,2.07,222102.0,-47565.0,83888.0,-1.68,-17.64,37.77,13.47,138.0,-0.57,1.58,12.93,XY24,NTT,PAINTS
18,CERA,9475.0,-23.15,36.0,H-SC,1.53,140150.0,-35753.0,77769.0,-0.48,-20.33,55.49,23.89,149.0,-0.46,1.00,21.04,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.38,42.0,H-MC,6.23,102220.0,-29615.0,71840.0,-0.26,-22.46,70.28,32.03,98.0,-0.41,0.73,14.50,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,53.0,M-MC,12.38,228596.0,3634.0,164018.0,0.10,1.62,71.75,74.53,192.0,0.02,1.63,34.39,XY24,BTT,STEEL
32,HINDZINC,730.22,30.43,67.0,M-LC,6.23,213128.0,8052.0,104518.0,-0.32,3.93,49.04,54.89,52.0,0.08,1.52,28.97,X5K,ATH,METALS
14,BLUESTARCO,2080.00,10.55,51.0,H-MC,2.97,190930.0,26260.0,17069.0,0.87,15.95,8.94,26.31,89.0,1.54,1.36,25.20,X40N,NTT,AC
84,VOLTAS,1530.00,-2.34,48.0,H-MC,1.42,205020.0,13278.0,24479.0,0.01,6.92,11.94,19.69,99.0,0.54,1.46,14.19,XY25,NTT,AC
17,CAMS,5211.76,-6.27,44.0,H-SC,1.83,106464.0,4460.0,39466.0,-0.61,4.37,37.07,43.06,122.0,0.11,0.76,22.24,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,62.0,H-MC,13.72,115010.0,14506.0,14606.0,-0.07,14.43,12.70,28.96,79.0,0.99,0.82,53.30,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,70.0,H-MC,10.81,193850.0,14042.0,98379.0,0.00,7.81,50.75,62.52,88.0,0.14,1.38,44.56,XR,NTT,BANKS
32,HINDZINC,730.22,30.43,67.0,M-LC,6.23,213128.0,8052.0,104518.0,-0.32,3.93,49.04,54.89,52.0,0.08,1.52,28.97,X5K,ATH,METALS
37,INDIAMART,4810.62,-55.57,32.0,H-SC,9.32,124248.0,912.0,130709.0,-1.33,0.74,105.20,106.72,119.0,0.01,0.88,24.29,AR,ATH,MISC
86,WIPRO,420.00,-14.25,45.0,M-LC,5.71,150796.0,-149.0,109599.0,0.45,-0.10,72.68,72.51,53.0,-0.00,1.07,5.92,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.15,31.0,H-LC,9.95,280592.0,-35250.0,147900.0,0.19,-11.16,52.71,35.67,5.0,-0.24,2.00,1.15,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-17.90,34.0,H-LC,5.77,207390.0,-44378.0,97598.0,0.08,-17.63,47.06,21.13,27.0,-0.45,1.48,11.55,X40,ATH,PAINTS
31,HINDUNILVR,2922.00,-10.50,40.0,H-LC,8.00,251200.0,-5863.0,40996.0,-1.17,-2.28,16.32,13.67,24.0,-0.14,1.79,17.66,XY25,NTT,FMCG
42,ITC,452.00,-39.61,41.0,H-LC,2.16,195600.0,-4538.0,25428.0,-0.19,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
19,CIPLA,1662.74,-22.18,43.0,H-LC,3.90,207254.0,2754.0,20539.0,-0.02,1.35,9.91,11.39,10.0,0.13,1.48,10.42,X40N,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,1.03,57.0,H-LC,2.01,162455.0,-23640.0,71123.0,-0.08,-12.70,43.78,25.51,15.0,-0.33,1.16,19.72,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,41.0,H-LC,2.16,195600.0,-4538.0,25428.0,-0.19,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
48,LTIM,7230.2,-4.62,53.0,H-LC,2.35,236745.0,-9406.0,88614.0,-0.25,-3.82,37.43,32.18,16.0,-0.11,1.69,30.84,X200,ATH,IT
51,NESTLEIND,1377.0,-10.51,50.0,H-LC,3.03,275231.0,9805.0,46982.0,-0.47,3.69,17.07,21.40,11.0,0.21,1.96,10.58,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.29,53.0,H-LC,3.34,216622.0,6256.0,22529.0,0.99,2.97,10.40,13.68,37.0,0.28,1.54,19.99,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.15,31.0,H-LC,9.95,280592.0,-35250.0,147900.0,0.19,-11.16,52.71,35.67,5.0,-0.24,2.00,1.15,X40N,ATH,FMCG
74,TCS,4406.34,-29.97,44.0,H-LC,12.18,279650.0,-66306.0,134540.0,-0.45,-19.17,48.11,19.73,1.0,-0.49,1.99,3.00,X40,ATH,IT
42,ITC,452.00,-39.61,41.0,H-LC,2.16,195600.0,-4538.0,25428.0,-0.19,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
80,UNITDSPR,1644.00,-11.52,55.0,H-LC,6.89,232062.0,-2082.0,50706.0,-0.87,-0.89,21.85,20.77,86.0,-0.04,1.65,5.97,X40N,NTT,BREWERIES
40,INFY,2275.00,-19.96,45.0,H-LC,8.08,311704.0,-1184.0,172871.0,-0.85,-0.38,55.46,54.87,3.0,-0.01,2.22,6.25,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,1.03,57.0,H-LC,2.01,162455.0,-23640.0,71123.0,-0.08,-12.70,43.78,25.51,15.0,-0.33,1.16,19.72,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,41.0,H-LC,2.16,195600.0,-4538.0,25428.0,-0.19,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
19,CIPLA,1662.74,-22.18,43.0,H-LC,3.90,207254.0,2754.0,20539.0,-0.02,1.35,9.91,11.39,10.0,0.13,1.48,10.42,X40N,ATH,PHARMA
5,ASIANPAINT,3465.66,-17.90,34.0,H-LC,5.77,207390.0,-44378.0,97598.0,0.08,-17.63,47.06,21.13,27.0,-0.45,1.48,11.55,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.29,53.0,H-LC,3.34,216622.0,6256.0,22529.0,0.99,2.97,10.40,13.68,37.0,0.28,1.54,19.99,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.97,44.0,H-LC,12.18,279650.0,-66306.0,134540.0,-0.45,-19.17,48.11,19.73,1.0,-0.49,1.99,3.00,X40,ATH,IT
40,INFY,2275.00,-19.96,45.0,H-LC,8.08,311704.0,-1184.0,172871.0,-0.85,-0.38,55.46,54.87,3.0,-0.01,2.22,6.25,X40,BTT,IT
42,ITC,452.00,-39.61,41.0,H-LC,2.16,195600.0,-4538.0,25428.0,-0.19,-2.27,13.00,10.44,4.0,-0.18,1.39,3.17,X40,NTT,FMCG
83,VBL,671.64,-22.15,31.0,H-LC,9.95,280592.0,-35250.0,147900.0,0.19,-11.16,52.71,35.67,5.0,-0.24,2.00,1.15,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,49.0,H-LC,7.52,250608.0,-11011.0,130216.0,0.06,-4.21,51.96,45.57,7.0,-0.08,1.78,6.64,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,31.63,79334.0,-34215.0,74265.0,0.22,-30.13,93.61,35.27,268.0,-0.46,0.56,94.17,XR,NTT,HOTELS
49,MASFIN,398.61,-18.25,49.0,H-SC,11.51,92805.0,-5175.0,26774.0,0.39,-5.28,28.85,22.05,152.0,-0.19,0.66,34.90,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.00,62.0,H-MC,13.72,115010.0,14506.0,14606.0,-0.07,14.43,12.70,28.96,79.0,0.99,0.82,53.30,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,65.0,H-SC,13.51,131220.0,-11259.0,30784.0,-0.59,-7.90,23.46,13.70,163.0,-0.37,0.93,54.88,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-185.23,69.0,M-SC,12.61,147159.0,-2061.0,146247.0,2.23,-1.38,99.38,96.62,208.0,-0.01,1.05,59.26,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-185.23,69.0,M-SC,12.61,147159.0,-2061.0,146247.0,2.23,-1.38,99.38,96.62,208.0,-0.01,1.05,59.26,XY25,NTT,FINANCE
69,SURYODAY,240.00,62.28,61.0,H-SC,6.64,148196.0,-30875.0,96609.0,-1.96,-17.24,65.19,36.71,135.0,-0.32,1.06,47.04,XR,NTT,BANKS
26,GREENPANEL,537.00,233.33,60.0,M-SC,3.70,148962.0,-34116.0,113092.0,1.01,-18.63,75.92,43.14,230.0,-0.30,1.06,40.12,XY24,NTT,MISC
32,HINDZINC,730.22,30.43,67.0,M-LC,6.23,213128.0,8052.0,104518.0,-0.32,3.93,49.04,54.89,52.0,0.08,1.52,28.97,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,70.0,H-MC,10.81,193850.0,14042.0,98379.0,0.00,7.81,50.75,62.52,88.0,0.14,1.38,44.56,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.10
1,25,43.35
2,50,75.13


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.07
LC    32.16
MC    23.77
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.10
X40      14.57
X40N     11.20
XR       10.97
XY25     10.40
AR        8.13
OX40N     7.69
X200      1.69
MH        1.59
X5K       1.52
SR        1.14
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.96
H-LC    25.63
H-MC    20.52
M-SC    13.70
M-LC     5.52
M-MC     2.95
L-SC     1.41
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.75,-6.27,41.29
IT,12.61,-18.79,83.13
FINANCE,10.55,-13.87,68.85
MISC,6.91,-18.30,83.86
BANKS,6.36,-11.63,70.35
PAINTS,5.55,-18.63,36.32
ELECTRICAL,5.37,-9.20,47.51
INSURANCE,3.83,-2.93,41.42
AC,3.51,1.82,14.62


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3205713.0,22
XR,1368688.0,14
AR,1186833.0,9
X40,794305.0,10
X40N,662241.0,9
OX40N,566106.0,10
XY25,476962.0,7
SR,262454.0,2
X5K,104518.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3722623.0,29
M-SC,1508554.0,17
H-MC,1272829.0,15
H-LC,1194964.0,15
M-LC,374939.0,4
M-MC,369021.0,2
L-SC,261738.0,3
L-MC,59907.0,1
L-LC,41873.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1197150.0      6
M-SC       XY24       847441.0      7
H-SC       AR         816801.0      5
           XR         799254.0      7
H-MC       XY24       565397.0      4
H-LC       X40        509477.0      5
M-MC       XY24       369021.0      2
H-SC       X40N       310692.0      4
           SR         262454.0      2
           OX40N      260864.0      4
H-LC       X40N       219145.0      3
H-MC       X40        211868.0      4
H-LC       AR         201339.0      2
H-MC       XY25       178335.0      2
L-SC       XR         172441.0      2
M-SC       AR         168693.0      2
M-LC       XY24       160822.0      2
M-SC       XY25       146247.0      1
           OX40N      144105.0      4
H-MC       X40N       132404.0      2
M-SC       XR         129108.0      2
H-LC       XY25       110507.0      3
M-LC       XR         109599.0      1
           X5K        104518.0      1
H-MC       XR          98379.0      1
L-SC       OX40N       89297.0      1
H-LC       X200        88614.0      1
H-SC       MH          75408.0      1
M-SC       X40         72960.0      1
H-MC       OX40N       71840.0      1
H-LC       XY24        65882.0      1
L-MC       XR          59907.0      1
L-LC       XY25        41873.0      1
H-MC       MH          14606.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
